In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os

In [ ]:
def gaussian_streak(image_size, center, length, width, direction, angle_deg, intensity):
    """
    Generate a synthetic Gaussian streak.
    """
    x0, y0 = center
    sigma_x = length / 2.355
    sigma_y = width / 2.355

    x = np.arange(0, image_size[1])
    y = np.arange(0, image_size[0])
    x, y = np.meshgrid(x, y)

    if direction == 'horizontal':
        x_rot = x - x0
        y_rot = y - y0
    elif direction == 'vertical':
        x_rot = x - x0
        y_rot = y - y0
    elif direction == 'diagonal':
        angle_rad = np.deg2rad(angle_deg)
        x_rot = (x - x0) * np.cos(angle_rad) + (y - y0) * np.sin(angle_rad)
        y_rot = -(x - x0) * np.sin(angle_rad) + (y - y0) * np.cos(angle_rad)
    else:
        raise ValueError("Invalid direction. Must be 'horizontal', 'vertical', or 'diagonal'.")

    streak = intensity * np.exp(-((x_rot)**2 / (2 * sigma_x**2) + (y_rot)**2 / (2 * sigma_y**2)))

    return streak

def is_non_overlapping(new_center, existing_centers, min_distance):
    """
    Check if the new center is non-overlapping with existing centers.
    """
    for center in existing_centers:
        distance = np.linalg.norm(np.array(new_center) - np.array(center))
        if distance < min_distance:
            return False
    return True

# Parameters
image_size = (256, 256)  # Set image size to 256x256
num_streaks = np.random.randint(1, 6)  # Randomly choose between 1 to 5 streaks

# Directory to save the images
save_dir = r"C:\Users\avsk1\OneDrive\Desktop\coding\more code\Multiple Streak"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Create a single image to hold all streaks
combined_image = np.zeros(image_size)

# List to keep track of centers of streaks
existing_centers = []

# Minimum distance to avoid overlap
min_distance = 50  # Adjust this value based on the expected size of the streaks

# Generate multiple non-overlapping streaks
for i in range(num_streaks):
    while True:
        # Random parameters for each streak
        center = (np.random.randint(0, image_size[0]), np.random.randint(0, image_size[1]))
        length = np.random.randint(30, 160)
        width = np.random.randint(5, 50)
        direction = np.random.choice(['horizontal', 'vertical', 'diagonal'])
        angle_deg = np.random.randint(0, 180) if direction == 'diagonal' else 0
        intensity = np.random.uniform(1.0, 3.0)

        # Check if the new center is non-overlapping
        if is_non_overlapping(center, existing_centers, min_distance):
            existing_centers.append(center)
            break

    # Generate streak
    streak = gaussian_streak(image_size, center, length, width, direction, angle_deg, intensity)

    # Add the streak to the combined image
    combined_image += streak

# Add Gaussian noise to the combined image
noise_level = 0.1
noise = np.random.normal(0, noise_level, image_size)
noisy_combined_image = combined_image + noise

# Save the synthetic FMA streaks in FITS format
hdu = fits.PrimaryHDU(combined_image)
hdu.writeto(os.path.join(save_dir, 'synthetic_fma_streaks_combined_02.fits'), overwrite=True)

hdu_noisy = fits.PrimaryHDU(noisy_combined_image)
hdu_noisy.writeto(os.path.join(save_dir, 'noisy_synthetic_fma_streaks_combined_02.fits'), overwrite=True)

# Plot the results
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(combined_image, cmap='gray')
plt.title(f'Combined Synthetic FMA Streaks ({num_streaks} streaks)')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(noisy_combined_image, cmap='gray')
plt.title(f'Noisy Combined Synthetic FMA Streaks ({num_streaks} streaks)')
plt.axis('off')

plt.tight_layout()
plt.show()